<a href="https://colab.research.google.com/github/Sattwikmaiti/EcommerceMachineModel/blob/second/ecommerceproductrecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import difflib #spelling mistakes from user. compare closest proximity to movies
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import csv
products = pd.read_csv('flipkart_com-ecommerce_sample.csv')
products.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
products.shape

(504, 15)

In [ ]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  504 non-null    object 
 1   crawl_timestamp          503 non-null    object 
 2   product_url              503 non-null    object 
 3   product_name             503 non-null    object 
 4   product_category_tree    503 non-null    object 
 5   pid                      503 non-null    object 
 6   retail_price             500 non-null    float64
 7   discounted_price         500 non-null    float64
 8   image                    503 non-null    object 
 9   is_FK_Advantage_product  503 non-null    object 
 10  description              503 non-null    object 
 11  product_rating           503 non-null    object 
 12  overall_rating           503 non-null    object 
 13  brand                    204 non-null    object 
 14  product_specifications   5

In [ ]:
products.dropna(inplace=True)
products.isnull().sum()

uniq_id                    0
crawl_timestamp            0
product_url                0
product_name               0
product_category_tree      0
pid                        0
retail_price               0
discounted_price           0
image                      0
is_FK_Advantage_product    0
description                0
product_rating             0
overall_rating             0
brand                      0
product_specifications     0
dtype: int64

In [ ]:
products.duplicated().sum()

0

In [ ]:
products['description'][0]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [ ]:
tfv = TfidfVectorizer()
products['description'] = products['description'].fillna('')


In [ ]:
tfv_matrix = tfv.fit_transform(products['description'])
tfv_matrix.shape

(504, 3491)

In [ ]:
sig = cosine_similarity(tfv_matrix,tfv_matrix)
sig[0]

array([1.00000000e+00, 7.77367313e-02, 1.22736539e-01, 9.44108838e-01,
       4.07448788e-02, 2.30760144e-02, 9.61475286e-01, 7.80031203e-02,
       7.33209170e-02, 9.33275228e-01, 6.14813967e-02, 1.35578627e-01,
       4.04579821e-02, 9.28487198e-01, 1.38294903e-01, 9.26177654e-01,
       7.78934466e-02, 8.81699224e-02, 1.35412112e-01, 7.77022062e-02,
       4.23442116e-02, 6.68409586e-01, 2.36284852e-01, 8.89481533e-02,
       2.26724312e-03, 1.90404137e-02, 1.32939753e-02, 9.88646700e-04,
       5.41204907e-02, 1.82616814e-02, 2.80936093e-02, 4.70716407e-02,
       4.47579620e-02, 4.47579620e-02, 3.91489401e-02, 1.93488007e-01,
       3.81834987e-02, 1.84024734e-01, 7.04050961e-02, 2.36831616e-01,
       8.29981422e-02, 3.14576758e-02, 4.16758160e-02, 4.33237087e-02,
       3.94542415e-01, 3.31356290e-01, 3.38076059e-01, 2.52908219e-02,
       3.53513430e-02, 2.36481517e-01, 3.29525904e-01, 4.18514514e-02,
       3.73402203e-02, 3.34363198e-01, 7.74239341e-02, 2.82871096e-01,
      

In [ ]:
indices = pd.Series(products.index,index=products['product_name']).drop_duplicates()
indices.head(20)

product_name
Alisha Solid Women's Cycling Shorts                                       0
FabHomeDecor Fabric Double Sofa Bed                                       1
AW Bellies                                                                2
Alisha Solid Women's Cycling Shorts                                       3
Sicons All Purpose Arnica Dog Shampoo                                     4
Eternal Gandhi Super Series Crystal Paper Weights  with Silver Finish     5
Alisha Solid Women's Cycling Shorts                                       6
FabHomeDecor Fabric Double Sofa Bed                                       7
dilli bazaaar Bellies, Corporate Casuals, Casuals                         8
Alisha Solid Women's Cycling Shorts                                       9
Ladela Bellies                                                           10
Carrel Printed Women's                                                   11
Sicons All Purpose Tea Tree Dog Shampoo                                  12

In [ ]:
def product_recommendation(title,sig=sig):
    indx = indices[title]

    #getting pairwise similarity scores
    sig_scores = list(enumerate(sig[indx]))

    # #sorting products
    # sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    #10 most similar products score
    sig_scores = sig_scores[1:11]

    #product indexes
    product_indices = [i[0] for i in sig_scores]

    #Top 10 most similar products
    return products['product_name'].iloc[product_indices]

In [ ]:
indices.head(100)

product_name
Alisha Solid Women's Cycling Shorts                                    0
FabHomeDecor Fabric Double Sofa Bed                                    1
AW Bellies                                                             2
Alisha Solid Women's Cycling Shorts                                    3
Sicons All Purpose Arnica Dog Shampoo                                  4
                                                                      ..
Reckler Slim Fit Men's Jeans                                          95
Quechua Arpenaz Novadry Boots                                         96
Colat COLAT_MW20 Sheen Analog Watch  - For Men, Women, Boys, Girls    97
Steppings Trendy Boots                                                98
Rochees RW38 Analog Watch  - For Boys                                 99
Length: 100, dtype: int64

In [ ]:
n=input("Enter the name of the product: ")
print("\nTop Recommended products are: \n")
print(product_recommendation(n))

Enter the name of the product: AW Bellies

Top Recommended products are: 

1                   FabHomeDecor Fabric Double Sofa Bed
2                                            AW Bellies
3                   Alisha Solid Women's Cycling Shorts
4                 Sicons All Purpose Arnica Dog Shampoo
5     Eternal Gandhi Super Series Crystal Paper Weig...
6                   Alisha Solid Women's Cycling Shorts
7                   FabHomeDecor Fabric Double Sofa Bed
8     dilli bazaaar Bellies, Corporate Casuals, Casuals
9                   Alisha Solid Women's Cycling Shorts
10                                       Ladela Bellies
Name: product_name, dtype: object
